# Homework Set 4

In [2]:
%pylab inline 
import pandas as pd
# import inst, fmt

Populating the interactive namespace from numpy and matplotlib


## Problem 1: 

Implement the Thomas algorithm for solving tridiagonal linear systems:

Do your own reading and research on <a href=http://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm>Thomas algorithm</a> and implement and test it in Python.

In [3]:
def solve_tridiagonal_matrix(matrix, d):
    _, n = matrix.shape
    a, b, c, x = (np.empty(n) for _ in range(4))
    c_p, d_p = [None for _ in range(n)], [None for _ in range(n)]
    for i in range(n):
        if i > 0:
            a[i] = matrix[i, i-1]
        if i < n - 1:
            c[i] = matrix[i, i+1]
        b[i] = matrix[i, i]
    for i in range(n-1):
        if i > 0:
            c_p[i] = c[i] / (b[i] - a[i] * c_p[i-1])
        else:
            c_p[i] = c[i] / b[i]
    for i in range(n):
        if i > 0:
            d_p[i] = (d[i] - a[i]*d_p[i-1]) / (b[i] - a[i] * c_p[i-1])
        else:
            d_p[i] = d[i] / b[i]
    x[n-1] = d_p[n-1]
    for i in range(n-2, -1, -1):
        x[i] = d_p[i] - c_p[i] * x[i+1]
    return x

In [4]:
def test_solve_tridiagonal_matrix(matrix, d):
    solution = solve_tridiagonal_matrix(matrix, d)
    assert np.allclose(matrix.dot(solution), d) is True

In [5]:
matrix = np.matrix([[1, 2, 0], [3, 4, 5], [0, 6, 7]])
test_solve_tridiagonal_matrix(matrix, np.array([8, 9, 10]))

## Problem 2

Given the following benchmark swap quotes for USD, suppose all swap coupons are paid semi-annually:

In [8]:
mats = np.array([1, 2, 3, 5, 7, 10, 12, 15, 20, 25]) * 1.
par = np.array([.042, .043, .047, .054, .057, .06, .061, .059, .056, .0555])

df_swap = pd.DataFrame(np.array([par]).T*100, columns=["Par Rate (%)"], 
                       index=map(lambda m: '%dY' % m, mats))
df_swap.T

,1Y,2Y,3Y,5Y,7Y,10Y,12Y,15Y,20Y,25Y
Par Rate (%),4.2,4.3,4.7,5.4,5.7,6.0,6.1,5.9,5.6,5.55


The swap pricers are provided by the python class ```Swap``` and the pricing function ```priceSwap```, both are defined in <a href=https://raw.githubusercontent.com/yadongli/nyumath2048/master/lib/swap.py>swap.py</a> under the folder nyumath2048/lib of the github repository.

Note the following assumptions of the swap pricer provided:
* it prices a receiver swap, you need to flip the sign to price a payer swap
* it ignores important details, such as day count conventions, do not use this code to trade!

The following are some sample codes of how to use the swap pricers:

In [7]:
from swap import Swap, priceSwap
import lin

# the pricing function if curve(t) is the cumulative yield, ie, -log(b(t))
def y2pv(swap, curve) :
    discf = lambda ts: np.exp(-curve(ts))
    return priceSwap(swap, discf)

# create the benchmark instruments
bm_swaps = {Swap(m, c, 2) : 0 for m, c in zip (mats, par)}

# price a single swap with a straight curve in y(t)
flat = lin.RationalTension(lbd = 5.)
flat.build(mats, mats*.04) # create a dummy curve of flat 0.04 rate

pvs = {swap.maturity : y2pv(swap, flat) for swap in bm_swaps.keys()}
print("PV by maturity")
print("\n".join(["%.2g Y : %.4g" % (m, v) for m, v in pvs.items()]))

ModuleNotFoundError: No module named 'lin'

1. Explain what is the purpose of the +1e-6 in the first line of the ```priceSwap``` function
2. Use the market data above to bootstrap the IR curve by interpolating the cumulative yield $y(t) = -\log(b(t))$ with tension spline, where $b(t)$ is the discount factor (i.e., the price of risk free zero coupon bonds). After bootstrapping the curve, re-price the benchmark instruments using the bootstrapped curve, and compute the L-2 norm of the absolute pricing errors. Show how the pricing error of the bootstrapped curve depend on the tension parameter $\lambda$.
3. Using the iteration technique to reduce the L-2 norm of the error below 1bps, how many iteration is needed for the error to go below 1bps?
4. From the curve built in the previous step, compute both the $y(t)$ and the instantaneous forward rate $f(t) = -\frac{1}{b(t)}\frac{d b(t)}{d t} = \frac{d y(t)}{dt}$, and show how their shapes change with the tension parameter $\lambda$. In addition, compare the changes in the instantaneous forward rates with 1bps change in the 5Y par swap rate. You can use the ```deriv()``` method in the Tension spline class. 
6. Repeat the previous step, but directly interpolating the forward rates $f(t)$, and comment on whether it is suitable in practice.
5. [extra credit] Build the curve by interpolating the zero rate with tension spline instead, i.e, $r(t) = \frac{y(t)}{t}$, compare how the forward rates shape and perturbations differ from those obtained in step 4, and comment on the pro and cons between interpolating $r(t)$ and $y(t)$.
7. [extra credit] write your own bootstrap function instead of calling those provided by the class library

Hint: 
* write your code in a modular and reusable way; you can re-use the code provided as part of the class lecture, but you get extra credit if you can write your own bootstrapping. The source code of the inst package is hidden from you, but you can figure out how to use them by reading their docstring with ```help()```, please use the python codes in the lecture slides as examples.
* be careful with the choice of boundary for the root search, allow negative rates often helps stabilizing the bootstrapping, even though it does not make economic sense
* in step 5, you may have to mix the old and new results to stabilize the iteration, (e.g., use ```mixf=0.5``` in the ```inst.iterboot``` function, or your own)